In [136]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import hvplot.pandas
%matplotlib inline

In [137]:
# Load .env enviroment variables
load_dotenv()

True

In [138]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [139]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-09-01', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-11-09', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL"]
# Set timeframe to "1Minute" for Alpaca API
timeframe = "15min"


In [140]:
### Creating Alpaca Dataframe
df_tickers=alpaca.get_bars(
        tickers,
        timeframe,
        start=start_time,
        end=end_time
    ).df
df_tickers

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-09-01 08:00:00+00:00,188.28,188.32,188.15,113,188.3200,4906,188.224731,AAPL
2023-09-01 08:15:00+00:00,188.23,188.23,188.23,14,188.2300,2080,188.229971,AAPL
2023-09-01 08:30:00+00:00,188.25,188.25,188.24,21,188.2400,695,188.247496,AAPL
2023-09-01 08:45:00+00:00,188.23,188.28,188.23,42,188.2500,3226,188.256122,AAPL
2023-09-01 09:00:00+00:00,188.43,188.44,188.28,121,188.2800,7652,188.349265,AAPL
...,...,...,...,...,...,...,...,...
2023-11-08 23:45:00+00:00,182.60,182.60,182.49,129,182.5898,8714,182.583517,AAPL
2023-11-09 00:00:00+00:00,182.51,182.72,182.51,146,182.6000,11604,182.603182,AAPL
2023-11-09 00:15:00+00:00,182.52,182.55,182.48,77,182.5100,3266,182.509584,AAPL


In [141]:
### Defining function to calculate MACD and Signal Line
def calculate_macd(df_tickers = df_tickers, short_window=12, long_window=26):
        df_tickers['ShortEMA'] = df_tickers['close'].ewm(span=short_window, adjust=False).mean()
        df_tickers['LongEMA'] = df_tickers['close'].ewm(span=long_window, adjust=False).mean()
        df_tickers['MACD'] = df_tickers['ShortEMA'] - df_tickers['LongEMA']
        df_tickers['SignalLine'] = df_tickers['MACD'].ewm(span=9, adjust=False).mean()
        df_tickers['histogram'] = df_tickers['MACD'] - df_tickers['SignalLine']
        return df_tickers

In [142]:
### Calculating the MACD Line
calculate_macd(df_tickers)

,close,high,low,trade_count,open,volume,vwap,symbol,ShortEMA,LongEMA,MACD,SignalLine,histogram
timestamp,,,,,,,,,,,,,
2023-09-01 08:00:00+00:00,188.28,188.32,188.15,113,188.3200,4906,188.224731,AAPL,188.280000,188.280000,0.000000,0.000000,0.000000
2023-09-01 08:15:00+00:00,188.23,188.23,188.23,14,188.2300,2080,188.229971,AAPL,188.272308,188.276296,-0.003989,-0.000798,-0.003191
2023-09-01 08:30:00+00:00,188.25,188.25,188.24,21,188.2400,695,188.247496,AAPL,188.268876,188.274348,-0.005473,-0.001733,-0.003740
2023-09-01 08:45:00+00:00,188.23,188.28,188.23,42,188.2500,3226,188.256122,AAPL,188.262895,188.271063,-0.008168,-0.003020,-0.005149
2023-09-01 09:00:00+00:00,188.43,188.44,188.28,121,188.2800,7652,188.349265,AAPL,188.288603,188.282836,0.005767,-0.001263,0.007029
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-08 23:45:00+00:00,182.60,182.60,182.49,129,182.5898,8714,182.583517,AAPL,182.679231,182.618511,0.060720,0.108391,-0.047670
2023-11-09 00:00:00+00:00,182.51,182.72,182.51,146,182.6000,11604,182.603182,AAPL,182.653196,182.610473,0.042723,0.095257,-0.052534
2023-11-09 00:15:00+00:00,182.52,182.55,182.48,77,182.5100,3266,182.509584,AAPL,182.632704,182.603771,0.028933,0.081992,-0.053060


In [143]:
### Using a loop to Calculate buy and sell signals based on MACD Line indicator and defining the total cashflow for that period. This shows if we made a profit or not
data_frame = df_tickers  
position = None 
buy_price = 0
total_buy_amount = 0
total_sell_amount = 0
sell_price = 0
signals = []
for i in range(1, len(data_frame)):
        if data_frame['MACD'][i] > data_frame['SignalLine'][i] and data_frame['MACD'][i - 1] <= data_frame['SignalLine'][i - 1]:
            # if position is None:
                position = 'buy'
                buy_price = data_frame['close'][i]
                total_buy_amount += buy_price
                signals.append((data_frame.index[i], 'Buy', buy_price))
        elif data_frame['MACD'][i] < data_frame['SignalLine'][i] and data_frame['MACD'][i - 1] >= data_frame['SignalLine'][i - 1]:
            # if position == 'buy':
                position = 'sell'
                sell_price = data_frame['close'][i]
                total_sell_amount += sell_price
                signals.append((data_frame.index[i], 'Sell', sell_price))
        
profit = total_sell_amount - total_buy_amount

In [144]:
#printing total buy and total sell 
print("Total Buy Amount:", total_buy_amount)
print("Total Sell Amount:", total_sell_amount)
print ("Profit:", (profit))

Total Buy Amount: 19491.3725
Total Sell Amount: 19669.7165
Profit: 178.34399999999732


In [145]:
Close_plot = df_tickers['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='Closing Price AAPL',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

MACD_plot = df_tickers['MACD'].hvplot.line(    
    x='timestamp', 
    y='MACD',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.2f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Signal_plot = df_tickers['SignalLine'].hvplot.line(    
    x='timestamp', 
    y='SignalLine',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.2f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)

Histogram_plot = df_tickers['histogram'].hvplot.bar(    
    x='timestamp', 
    y='histogram',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='MACD-Signal', 
    xlabel='timestamp',
    title='MACD Line - Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.2f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)



In [146]:
Close_plot

:Curve   [timestamp]   (close)

In [147]:
plot = MACD_plot * Signal_plot
plot

:Overlay
   .Curve.MACD       :Curve   [timestamp]   (MACD)
   .Curve.SignalLine :Curve   [timestamp]   (SignalLine)

In [148]:
Histogram_plot

:Bars   [timestamp]   (histogram)

In [149]:
#printing total buy and total sell 
print("Total Buy Amount:", total_buy_amount)
print("Total Sell Amount:", total_sell_amount)
print ("Profit:", (profit))

Total Buy Amount: 19491.3725
Total Sell Amount: 19669.7165
Profit: 178.34399999999732


In [150]:
signals

[(Timestamp('2023-09-01 08:15:00+0000', tz='UTC'), 'Sell', 188.23),
 (Timestamp('2023-09-01 09:00:00+0000', tz='UTC'), 'Buy', 188.43),
 (Timestamp('2023-09-01 14:45:00+0000', tz='UTC'), 'Sell', 188.4),
 (Timestamp('2023-09-01 16:45:00+0000', tz='UTC'), 'Buy', 189.175),
 (Timestamp('2023-09-01 17:30:00+0000', tz='UTC'), 'Sell', 188.915),
 (Timestamp('2023-09-01 17:45:00+0000', tz='UTC'), 'Buy', 189.17),
 (Timestamp('2023-09-01 18:15:00+0000', tz='UTC'), 'Sell', 188.7201),
 (Timestamp('2023-09-01 19:45:00+0000', tz='UTC'), 'Buy', 189.47),
 (Timestamp('2023-09-01 22:00:00+0000', tz='UTC'), 'Sell', 189.21),
 (Timestamp('2023-09-05 11:30:00+0000', tz='UTC'), 'Buy', 187.85),
 (Timestamp('2023-09-05 19:00:00+0000', tz='UTC'), 'Sell', 189.56),
 (Timestamp('2023-09-06 11:15:00+0000', tz='UTC'), 'Buy', 188.7),
 (Timestamp('2023-09-06 13:30:00+0000', tz='UTC'), 'Sell', 186.915),
 (Timestamp('2023-09-06 17:45:00+0000', tz='UTC'), 'Buy', 182.9699),
 (Timestamp('2023-09-07 08:00:00+0000', tz='UTC'),